In [ ]:
import numpy as np
import os
import pickle
import torch
from iopath.common.file_io import g_pathmgr

import slowfast.utils.checkpoint as cu
import slowfast.utils.distributed as du
import slowfast.utils.logging as logging
import slowfast.utils.misc as misc
import slowfast.visualization.tensorboard_vis as tb
from slowfast.datasets import loader
from slowfast.models import build_model
from slowfast.utils.meters import AVAMeter, TestMeter


In [ ]:
logger = logging.get_logger(__name__)

In [ ]:
@torch.no_grad()
def perform_test(test_loader, model, test_meter, cfg, writer=None):
    
    # Enable eval mode.
    model.eval()
    test_meter.iter_tic()
    
    for cur_iter, (inputs, labels, video_idx, meta) in enumerate(test_loader):
        test_meter.data_toc()
        
        # Perform the forward pass.
        preds = model(inputs)

        # Gather all the predictions across all the devices to perform ensemble.
        if cfg.NUM_GPUS > 1:
            preds, labels, video_idx = du.all_gather(
                [preds, labels, video_idx]
            )
        if cfg.NUM_GPUS:
            preds = preds.cpu()
            labels = labels.cpu()
            video_idx = video_idx.cpu()

        test_meter.iter_toc()
        # Update and log stats.
        test_meter.update_stats(
            preds.detach(), labels.detach(), video_idx.detach()
        )
        test_meter.log_iter_stats(cur_iter)
        
        test_meter.iter_tic()
        test_meter.finalize_metrics()
        return test_meter
        